In [1]:
from google.colab import drive

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
# import the libraries as shown below

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50
#from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [5]:
IMAGE_SIZE = [224, 224]

train_path = 'Datasets/Train'
valid_path = 'Datasets/Test'

In [6]:
resnet = ResNet50(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

94773248/94765736 [==============================] - 39s 0us/step


In [7]:
for layer in resnet.layers:
    layer.trainable = False

In [8]:
folders = glob('/content/drive/My Drive/Datasets/Train/*')

In [9]:
folders

[]

In [ ]:
test_folders = glob('/content/drive/My Drive/Datasets/Test/*')

In [ ]:
test_folders

['/content/drive/My Drive/Datasets/Test/lamborghini',
 '/content/drive/My Drive/Datasets/Test/audi',
 '/content/drive/My Drive/Datasets/Test/mercedes']

In [ ]:
x = Flatten()(resnet.output)

In [ ]:
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=resnet.input, outputs=prediction)

In [ ]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
____________________________________________________________________________________________

In [ ]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
training_set = train_datagen.flow_from_directory('/content/drive/My Drive/Datasets/Train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 64 images belonging to 3 classes.


In [ ]:
test_set = test_datagen.flow_from_directory('/content/drive/My Drive/Datasets/Test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 58 images belonging to 3 classes.


In [ ]:
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=50,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

Epoch 1/50
2/2 [==============================] - 2s 795ms/step - loss: 7.7295 - accuracy: 0.2969 - val_loss: 9.4788 - val_accuracy: 0.1552
Epoch 2/50
2/2 [==============================] - 1s 407ms/step - loss: 7.4852 - accuracy: 0.3438 - val_loss: 6.6849 - val_accuracy: 0.3276
Epoch 3/50
2/2 [==============================] - 1s 408ms/step - loss: 6.0901 - accuracy: 0.5000 - val_loss: 4.9021 - val_accuracy: 0.5172
Epoch 4/50
2/2 [==============================] - 1s 399ms/step - loss: 6.4030 - accuracy: 0.3594 - val_loss: 1.6629 - val_accuracy: 0.5345
Epoch 5/50
2/2 [==============================] - 1s 402ms/step - loss: 1.9003 - accuracy: 0.5469 - val_loss: 5.7834 - val_accuracy: 0.1724
Epoch 6/50
2/2 [==============================] - 1s 395ms/step - loss: 4.7213 - accuracy: 0.4219 - val_loss: 6.6341 - val_accuracy: 0.1552
Epoch 7/50
2/2 [==============================] - 1s 399ms/step - loss: 3.3032 - accuracy: 0.5312 - val_loss: 3.9003 - val_accuracy: 0.3276
Epoch 8/50
2/2 [====

In [ ]:
from tensorflow.keras.models import load_model

model.save('model_resnet50.h5')

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

In [ ]:
model=load_model('model_resnet50.h5')

In [ ]:
img=image.load_img('/content/drive/My Drive/Datasets/Test/lamborghini/15.jpg',target_size=(224,224))

x=image.img_to_array(img)
# x

In [ ]:
x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)
img_data.shape

(1, 224, 224, 3)

In [ ]:
model.predict(img_data)

array([[1.9519612e-15, 5.3459535e-19, 1.0000000e+00]], dtype=float32)

In [ ]:
a=np.argmax(model.predict(img_data), axis=1)

In [ ]:
a

array([2])